In [ ]:
import numpy as np
import astropy
import scipy
import matplotlib.pyplot as plt
import emcee
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
import pymc
import corner.corner as corner
import astroML
import sklearn
import os
import time
import ultranest
import math

In [ ]:
# read in data
data = np.load("/home/kaldorme/classes/astr_8070_s25/coursework/student_folders/marykaldor/spectral_decomp/fit_uncert/56449.npy")
w = data[0]
fl = data[1]
uncert = data[2]

In [ ]:
def disk(values):
    params = ['Q1',\
          'Q2',\
          'XIB',\
          'ANGI',\
          'XI1',\
          'XI2',\
          'BROAD']
          #'T0',\
          #'ETA',\
          #'ANGLAM',\
          #'AMP',\
          #'NARMS',\
          #'AOBS',\
          #'PITCH',\
          #'WIDTH']
          #'XISPIN',\
          #'XISPOUT']
    values[2] = str(float(values[4])+float(values[2]))
    values[5] = str(float(values[4])+float(values[5]))
    # Open old file in read mode and create new file in write mode
    fin = open("./parfile_original.dat", "r")
    fout = open("./parfile.dat", "w")

    # Setting up list iterations
    i = 0

    # Setting up range over which to iterate
    j = 0
    k = len(values)-1

    # Go through text and only change given parameter's value
    # Everything else just gets rewritten to the new file
    while j<k:
        for line in fin:
            if params[j] in line:
                x = line.split()
                if params[j] == x[1]:
                    if j==11:
                        # Forcing to write the number of spiral arms in the disk as an integer
                        fout.write(line.replace(x[0], str(int(values[j]))))
                    else:
                        fout.write(line.replace(x[0], str(values[j])))
                    if params[j] == params[-1]:
                        pass
                    else:
                        j += 1
                else:
                    fout.write(line)
            else:
                fout.write(line)
    
    fin.close()
    fout.close()

    os.system("./calc_profile_wave_pkg.x")
    wavelength = []
    flambda = []
    f = open('model.dat')
    lines = f.readlines()

    # Gets rid of pound symbol lines
    for line in lines:
        if line[0] == "#":
            pass
        else:
            x, y = line.split()
            y = y.strip("\n")
            x = float(x)
            y = float(y)
            wavelength.append(x)
            flambda.append(y)

    wavelength = np.array(wavelength)
    flambda = np.array(flambda)

    '''
    plt.figure(figsize=(16,12))
    plt.plot(wavelength, flambda, label="Fit")
    plt.plot(w,fl, label="Initial")
    plt.legend(fontsize=30)
    plt.xlabel("Wavelength ($\AA$)", fontsize=30)
    plt.ylabel("Flux (Normalized)", fontsize=30)
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.show()
    '''
    
    return wavelength, flambda

# compute the data log-likelihood given a model
def logL(values, model=disk):
    #print("xi1", values[4], "xispin", values[15], "xispout", values[16], "xi2", values[5])
    #if values[4]>values[5]:
        #loglike = -1e100
    y_fit = model(values)
    y = fl
    u = uncert
    loglike = sum(sum(scipy.stats.norm.logpdf(*args) for args in zip(y, y_fit, u)))
    return loglike

In [ ]:
params_nostring = ['Q1',\
          'Q2',\
          'XIB',\
          'ANGI',\
          'XI1',\
          'XI2',\
          'BROAD']
          #'T0',\
          #'ETA',\
          #'ANGLAM',\
          #'AMP',\
          #'NARMS',\
          #'AOBS',\
          #'PITCH',\
          #'WIDTH']
          #'XISPIN',\
          #'XISPOUT']

values_nostring = ['1.0',\
          '0.8',\
          '1000',\
          '30',\
          '1000',\
          '3000',\
          '1100']
          #'1e-7',\
          #'0.',\
          #'15',\
          #'4',\
          #'1',\
          #'124',\
          #'60',\
          #'62']
          #'0',\
          #'0']

In [ ]:
initial_w, initial_fl = disk(values_nostring)

In [ ]:
def my_prior_transform(cube):
    params = cube.copy()
    
    # Q1 uniform prior
    lo = 0.
    hi = 1.5
    params[0] = cube[0] * (hi - lo) + lo

    # Q2 uniform prior
    lo = 0.
    hi = 1.5
    params[1] = cube[1] * (hi - lo) + lo
    
    # DELTA_XIB uniform prior
    lo = 0
    hi = 10000
    params[2] = cube[2] * (hi - lo) + lo

    # ANGI uniform prior
    lo = 20.
    hi = 50.
    params[3] = cube[3] * (hi - lo) + lo

    # XI1 uniform prior
    lo = 100
    hi = 10000
    params[4] = cube[4] * (hi - lo) + lo
    
    # XI2 uniform prior
    #lo = 100
    #hi = 20000
    #params[5] = cube[5] * (hi - lo) + lo

    # DELTA_XI uniform prior
    lo = 100.
    hi = 10000.
    params[5] = cube[5] * (hi - lo) + lo

    # BROAD uniform prior
    lo = 100.
    hi = 2000.
    params[6] = cube[6] * (hi - lo) + lo

    '''
    
    # T0 uniform prior
    lo = 0.
    hi = 0.001
    params[7] = cube[7] * (hi - lo) + lo

    # ETA uniform prior
    lo = -0.1
    hi = 1.
    params[8] = cube[8] * (hi - lo) + lo

    # ANGLAM uniform prior
    lo = 0.
    hi = 90.
    params[9] = cube[9] * (hi - lo) + lo

    # AMP uniform prior
    lo = 0.
    hi = 10.
    params[10] = cube[10] * (hi - lo) + lo

    # NARMS uniform prior
    lo = 0
    hi = 10
    params[11] = cube[11] * (hi - lo) + lo

    # AOBS uniform prior
    lo = -360.
    hi = 360.
    params[12] = cube[12] * (hi - lo) + lo

    # PITCH uniform prior
    lo = -90.
    hi = 90.
    params[13] = cube[13] * (hi - lo) + lo

    # WIDTH uniform prior
    lo = 0.
    hi = 90.
    params[14] = cube[14] * (hi - lo) + lo
    '''

    '''
    # XISPIN uniform prior
    lo = 100
    hi = 20000
    params[15] = cube[15] * (hi - lo) + lo

    # XISPOUT uniform prior
    lo = 100
    hi = 20000
    params[16] = cube[16] * (hi - lo) + lo
    '''

    return params

In [ ]:
sampler = ultranest.ReactiveNestedSampler(params_nostring, logL, my_prior_transform)

In [ ]:
result = sampler.run(min_num_live_points=1000)
sampler.print_results()

In [ ]:
ultranest_final_params = sampler.results['posterior']['mean']
#ultranest_final_params = [math.trunc(i) for i in ultranest_final_params]
disk(ultranest_final_params)
ultranest_final_params

In [ ]:
from ultranest.plot import cornerplot
cornerplot(result)

In [ ]:
'''
logz_wind = result['logz']
logz_wind
'''

In [ ]:
logz_nowind = result['logz']
logz_nowind

In [ ]:
bayes = logz_wind/logz_nowind

In [ ]:
bayes

In [ ]:
bayes2 = np.exp(logz_wind-logz_nowind)
bayes2